# 밑준비하기

In [1]:
import os, glob
import logging

from pprint import pprint
from tqdm import tqdm
import json
# import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅한다
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 512  # Maximum length of the input to the Encoder
MIN_TARGET_LENGTH = 5  # Minimum length of the output by Decoder
MAX_TARGET_LENGTH = 128  # Maximum length of the output by Decoder
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# HF model hub에서 가져올 모델 이름
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

## Tensorflow 환경에서 TPU를 사용하도록 설정하기

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.TPUStrategy(tpu)

print(f"Available number of replicas: {strategy.num_replicas_in_sync}")

Available number of replicas: 8


# TFRecord 생성하기

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
print(_bytes_feature(b'hello'))
print(_bytes_feature('사과'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(False))
print(_int64_feature(1))

bytes_list {
  value: "hello"
}

bytes_list {
  value: "\354\202\254\352\263\274"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 0
}

int64_list {
  value: 1
}



In [ ]:
print(b'\354\202\254\352\263\274'.decode('utf-8'))

사과


In [ ]:
feature = _bytes_feature(u'사과'.encode('utf-8'))

feature.SerializeToString()

b'\n\x08\n\x06\xec\x82\xac\xea\xb3\xbc'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ds_dir = './drive/MyDrive/Datasets/Texts/09.literature/20per/'
ds_path = glob.glob(ds_dir + '*.json')
len(ds_path)

4800

In [ ]:
def make_example(id, document, summary):
    # Create a dictionary with features that may be relevant.
    feature = {'id': _bytes_feature(id.encode('utf-8')),
               'document': _bytes_feature(document.encode('utf-8')),
               'summary': _bytes_feature(summary.encode('utf-8'))}

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
# TFRecord에 저장된 데이터의 예시
for path_ in [ds_path[0]]:

    with open(path_, 'r') as f:

        doc = json.load(f)

        doc_id = doc["Meta(Acqusition)"]["doc_id"]
        document = doc['Meta(Refine)']['passage']
        summary = doc['Annotation']['summary1']

        tf_example = make_example(doc_id, document, summary)
tf_example.SerializeToString()

In [ ]:
def main(dataset_path, output_path):
    # json에서 필요한 정보들을 추출해서 list에 담기
    samples = []
    print("Reading data list...")
    for path_ in dataset_path:

        with open(path_, 'r') as f:

            doc = json.load(f)

            doc_id = doc["Meta(Acqusition)"]["doc_id"]
            document = doc['Meta(Refine)']['passage']
            summary = doc['Annotation']['summary1']
            samples.append((doc_id, document, summary))

    # list에 담긴 모든 정보들을 형 변환해서 tfrecords로 저장
    print("Writing tfrecord file...")
    with tf.io.TFRecordWriter(output_path) as writer:
        for doc_id, document, summary in tqdm(samples):
            tf_example = make_example(doc_id, document, summary)
            writer.write(tf_example.SerializeToString())

In [ ]:
os.makedirs('./tfrecord', exist_ok=True)

In [ ]:
main(ds_path, './tfrecord/literature.tfrecords')

Reading data list...
Writing tfrecord file...


100%|██████████| 4800/4800 [00:00<00:00, 7991.91it/s]


# TFRecord 불러오기

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
def _parse_function(example_proto):

    name_to_features = {'id': tf.io.FixedLenFeature([], tf.string),
                        'document': tf.io.FixedLenFeature([], tf.string),
                        'summary': tf.io.FixedLenFeature([], tf.string)}

    example = tf.io.parse_single_example(example_proto, name_to_features)

    for name in list(example.keys()):
        t = example[name]
        t = tf.cast(t, tf.string)
        example[name] = t

    return example

In [ ]:
def load_tfrecord_dataset(tfrecord_name, batch_size, shuffle=True, buffer_size=10240):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()

    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)

    name_to_features = {'id': tf.io.FixedLenFeature([], tf.string),
                        'document': tf.io.FixedLenFeature([], tf.string),
                        'summary': tf.io.FixedLenFeature([], tf.string)}


    dataset = raw_dataset.map(
        _parse_function,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )



    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
tfr_path = 'gs://ohsori-tfrecord/tfrecord/literature.tfrecords'
tf_record = load_tfrecord_dataset(tfr_path, batch_size=BATCH_SIZE, shuffle=False)
tf_record = iter(tf_record)

In [ ]:
[summary.decode('utf-8') for summary in next(tf_record)['summary'].numpy()]

['한치각은 남주사와 일지매에 대한 여러 가지 충동에 때문에 용이히 떠나지 않았다.',
 '효남이는 벌판에 혼자 남아 목장에서 소동이 난 줄도 모르고 땅바닥에 엎드려 울고만 있었다.',
 '엘리어트는 일차 대전을 분기로 한 현대에서는 시가 흥성할 때 비평이 위축되고 비평이 흥성할 때 시가 위축된다는 사실을 추리했다.',
 '그 지방 장관이 마차에 치여 즉사한 사람 중에 있었는데 이는 그 사건의 결과를 중대화시켜 그를 꼭 벌해야 했다.',
 '어머니가 몸부림치며 매달리자 덕쇠는 어머니의 머리와 몸뚱이를 통째로 밀어버렸다.',
 '아버지가 하인에게 맏아들을 매질하라 불호령을 내렸으나 아들을 지지 않고 칼집에 손을 대었고 어머니와 누이들이 말리자 그 즉시로 맏아들은 집을 나가고 만 것이다.',
 '이튿날 아침 김통역은 문밖에서 기다린 삼 형제가 밤새 있던 여자와 영구히 살라는 간청에 국법 때문에 그리할 수 없다고 말했다.',
 '사로당은 남로당에 조건 없는 합당을 제의했지만 받아들여지지 않았고 나는 좌익의 합당이 중요하다고 생각하며 사로당의 해산이 내부 갈등을 중단시키는 최선의 방법이라고 생각한다.']

# Reference

TFRecord 생성 및 불러오기

- https://www.tensorflow.org/tutorials/load_data/tfrecord?hl=ko

- https://baekyeongmin.github.io/dev/tpu-recipe-2/

- https://velog.io/@riverdeer/TFRecord-%ED%8C%8C%EC%9D%BC-%EC%9D%BD%EA%B3%A0-%EC%93%B0%EA%B8%B0

